# Asking and Answering Questions on Stack Overflow
Stack Overflow has released official [How do I ask a good question?](https://stackoverflow.com/help/how-to-ask) and [How do I write a good answer?](https://stackoverflow.com/help/how-to-answer) guides.

Why do we care about the quality of the questions asked? Where does that put the pedagogical "There is no such thing as a bad question?"

Although there is a educational aspect to Stack Overflow, at the end of the day, it is a question and answer forum for developers to get and give help. If you are asking for help, you want to maximize your chances at getting it. If you are helping, you want to maximize your chances of being effective at helping. In this context, we could rephrase "good" questions to be questions likely to be answered, and "good" answers to be answers likely to be accepted.

Stack Overflow has the feature that

> Sometimes users encounter the following message when posting a question: 
>> 'This post does not meet our quality standards.'

>If you see this message, then your question was automatically blocked by the server. All new questions are subjected to a "minimum quality" filter that checks for some basic indicators of a good, complete question. Check to make sure that your question has the following:
A clear title.
A reasonable explanation of what your question is. Add as much detail as you can.
Any background research you've tried but wasn't enough to solve your problem.
Correct use of English spelling and grammar to the best of your ability.

However, everything they list above is quite subjective. Let's explore what features correlate with the votes and answers for a question.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from multiprocessing import Pool
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
import re

In [42]:
Questions = pd.read_csv('pythonquestions/Questions.csv', encoding="ISO-8859-1")
Answers = pd.read_csv('pythonquestions/Answers.csv', encoding="ISO-8859-1")
Tags = pd.read_csv('pythonquestions/Tags.csv', encoding="ISO-8859-1")

Questions.columns = ['QID', 'QuestionUserId', 'QuestionCreateDate', 'QuestionScore', 'QuestionTitle', 'QuestionBody']
Answers.columns = ['AID','AnswerUserId', 'AnswerCreateDate', 'ParentId', 'AnswerScore', 'AnswerBody']
Tags.columns = ['TID', 'Tag']

In [43]:
print(Questions.shape)
Questions.head()

(607282, 6)


,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [44]:
print(Answers.shape)
Answers.head()

(987122, 6)


,AID,AnswerUserId,AnswerCreateDate,ParentId,AnswerScore,AnswerBody
0,497,50.0,2008-08-02T16:56:53Z,469,4,<p>open up a terminal (Applications-&gt;Utilit...
1,518,153.0,2008-08-02T17:42:28Z,469,2,<p>I haven't been able to find anything that d...
2,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...
3,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...
4,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B..."


In [45]:
print(Tags.shape)
Tags.head()

(1885078, 2)


,TID,Tag
0,469,python
1,469,osx
2,469,fonts
3,469,photoshop
4,502,python


In [46]:
print(Questions.isnull().sum())
print(Answers.isnull().sum())
print(Tags.isnull().sum())

QID                      0
QuestionUserId        6212
QuestionCreateDate       0
QuestionScore            0
QuestionTitle            0
QuestionBody             0
dtype: int64
AID                    0
AnswerUserId        5367
AnswerCreateDate       0
ParentId               0
AnswerScore            0
AnswerBody             0
dtype: int64
TID      0
Tag    443
dtype: int64


Looks like this data is pretty clean. The only nulls are for users (probably users that have deleted their accounts since posting), and tags (since these are optional).

### Question 1: How many users are engaged on Q, A, & Q&A?

In [47]:
User_id_inQ = Questions['QuestionUserId'].unique()
User_id_inA = Answers['AnswerUserId'].unique()
User_id_inBoth=set(User_id_inQ).intersection(User_id_inA)

In [48]:
print(str(len(User_id_inQ)) + ' users posting questions')
print(str(len(User_id_inA)) + ' users posting answers')
print(str(len(User_id_inBoth)) + ' users posting both')

213928 users posting questions
149177 users posting answers
63779 users posting both


In [49]:
# reduce memory and computation
# selected_ids = np.random.choice(range(Questions.shape[0]), 10000, replace=False)
# Questions = Questions.loc[selected_ids, :]

### Question 2: How many questions contain Python errors and tracebacks?

The following is to see what happens as I build up the regex to what Python error messages are formatted as:

In [51]:
print(
    Questions['QuestionBody'].str.contains("Error").sum(), 
    Questions['QuestionBody'].str.contains("Error:").sum(), 
    Questions['QuestionBody'].str.contains("Error:\s").sum(), 
    Questions['QuestionBody'].str.contains("[a-z]+Error:\s").sum(), 
    Questions['HasError'].sum()
)

118516 96055 95190 85601 85481


In [50]:
Questions['HasError'] = Questions['QuestionBody'].str.contains("[A-Z][a-z]+Error:\s")

In [52]:
print(str(Questions['HasError'].sum() / Questions['HasError'].count()) + " of questions have a full Python-formatted error message")

0.14075997641952173 of questions have a full Python-formatted error message


In [53]:
Questions['HasTraceback'] = Questions['QuestionBody'].str.contains("Traceback \(most recent call last\):")

In [54]:
print(str(Questions['HasTraceback'].sum() / Questions['HasTraceback'].count()) + " of questions have a full Python-formatted traceback")

0.06771977433877506 of questions have a full Python-formatted traceback


In [55]:
Questions.head()

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,False,False
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,False,False
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,False,False
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,False,False


### Overall number of votes per question

In [56]:
total_average_votes = Questions['QuestionScore'].sum() / len(Questions['QuestionScore'])
print('total average votes ' + str(total_average_votes))

total average votes 2.2831369940159596


### Question 3: Do questions with errors have more or less votes?

In [57]:
QuestionsWithError = Questions[Questions['HasError']]
hasError_average_votes = QuestionsWithError['QuestionScore'].sum() / len(QuestionsWithError['QuestionScore'])
print('error average votes ' + str(hasError_average_votes))
QuestionsWithOnlyError = Questions[(Questions['HasError'] & ~Questions['HasTraceback'])]
hasOnlyError_average_votes = QuestionsWithOnlyError['QuestionScore'].sum() / len(QuestionsWithOnlyError['QuestionScore'])
print('only error average votes ' + str(hasOnlyError_average_votes))

error average votes 1.7215053637650473
only error average votes 1.6573415912339209


### Question 4: Do questions with tracebacks have more or less votes?

In [58]:
QuestionsWithTraceback = Questions[Questions['HasTraceback']]
hasTraceback_average_votes = QuestionsWithTraceback['QuestionScore'].sum() / len(QuestionsWithTraceback['QuestionScore'])
print('traceback average votes ' + str(hasTraceback_average_votes))
QuestionsWithOnlyTraceback = Questions[(~Questions['HasError'] & Questions['HasTraceback'])]
hasOnlyTraceback_average_votes = QuestionsWithOnlyTraceback['QuestionScore'].sum() / len(QuestionsWithOnlyTraceback['QuestionScore'])
print('only traceback average votes ' + str(hasOnlyTraceback_average_votes))

traceback average votes 1.8253617021276596
only traceback average votes 1.8328611898016998


### Question 5: Do questions with errors and tracebacks have more or less votes?

In [59]:
QuestionsWithErrorAndTraceback = Questions[(Questions['HasError'] & Questions['HasTraceback'])]

In [60]:
QuestionsWithErrorAndTraceback = QuestionsWithError[QuestionsWithError['HasTraceback']]
hasErrorAndTraceback_average_votes = QuestionsWithErrorAndTraceback['QuestionScore'].sum() / len(QuestionsWithErrorAndTraceback['QuestionScore'])
print('error and traceback average votes ' + str(hasErrorAndTraceback_average_votes))

error and traceback average votes 1.8235169363146095


### Overall average number of answers per question

In [74]:
QA = Questions.merge(Answers, how='left', left_on='QID', right_on='ParentId')
QA.head(3)

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback,AID,AnswerUserId,AnswerCreateDate,ParentId,AnswerScore,AnswerBody
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...
1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...
2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,3040.0,457.0,2008-08-06T03:01:23Z,469.0,12.0,<p>Unfortunately the only API that isn't depre...


In [75]:
AnswersPerQuestion = QA.groupby('ParentId').count()
AnswersPerQuestion.sort_values('AnswerCreateDate', ascending=False).head()

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback,AID,AnswerUserId,AnswerCreateDate,AnswerScore,AnswerBody
ParentId,,,,,,,,,,,,,
550632.0,55,55,55,55,55,55,55,55,55,54,55,55,55
312443.0,45,45,45,45,45,45,45,45,45,45,45,45,45
36932.0,43,43,43,43,43,43,43,43,43,42,43,43,43
89228.0,42,42,42,42,42,42,42,42,42,42,42,42,42
18686860.0,37,37,37,37,37,37,37,37,37,37,37,37,37


In [76]:
total_average_answers = AnswersPerQuestion['AnswerCreateDate'].sum() / AnswersPerQuestion['AnswerCreateDate'].count()
print('total average answers ' + str(total_average_answers))

total average answers 1.8305868651689978


### Question 6: Do questions with errors have more or less answers?

In [77]:
AnswersPerQuestionWithError = QA[QA['HasError']].groupby('ParentId').count()
hasError_average_answers = AnswersPerQuestionWithError['AnswerCreateDate'].sum() / len(AnswersPerQuestionWithError['AnswerCreateDate'])
print('error average answers ' + str(hasError_average_answers))
AnswersPerQuestionWithOnlyError = QA[(QA['HasError'] & ~QA['HasTraceback'])].groupby('ParentId').count()
hasOnlyError_average_answers = AnswersPerQuestionWithOnlyError['AnswerCreateDate'].sum() / len(AnswersPerQuestionWithOnlyError['AnswerCreateDate'])
print('only error average answers ' + str(hasOnlyError_average_answers))

error average answers 1.6628310551046561
only error average answers 1.671052063080579


### Question 7: Do questions with tracebacks have more or less answers?

In [78]:
AnswersPerQuestionWithTraceback = QA[QA['HasTraceback']].groupby('ParentId').count()
hasTraceback_average_answers = AnswersPerQuestionWithTraceback['AnswerCreateDate'].sum() / len(AnswersPerQuestionWithTraceback['AnswerCreateDate'])
print('traceback average answers ' + str(hasError_average_answers))
AnswersPerQuestionWithOnlyTraceback = QA[(~QA['HasError'] & QA['HasTraceback'])].groupby('ParentId').count()
hasOnlyTraceback_average_answers = AnswersPerQuestionWithOnlyTraceback['AnswerCreateDate'].sum() / len(AnswersPerQuestionWithOnlyTraceback['AnswerCreateDate'])
print('only traceback average answers ' + str(hasOnlyTraceback_average_answers))

traceback average answers 1.6628310551046561
only traceback average answers 1.600452488687783


### Question 8: Do questions with errors and tracebacks have more or less answers?

In [79]:
AnswersPerQuestionWithErrorAndTraceback = QA[(QA['HasError'] & QA['HasTraceback'])].groupby('ParentId').count()
hasErrorAndTraceback_average_answers = AnswersPerQuestionWithErrorAndTraceback['AnswerCreateDate'].sum() / len(AnswersPerQuestionWithErrorAndTraceback['AnswerCreateDate'])
print('error traceback average answers ' + str(hasErrorAndTraceback_average_answers))

error traceback average answers 1.6495353224792118


## Preparing text for NLP

Let's also merge in the tags for the questions.

In [82]:
QAT = QA.merge(Tags, how='left', left_on='QID', right_on='TID')

In [83]:
QAT.head(100)

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback,AID,AnswerUserId,AnswerCreateDate,ParentId,AnswerScore,AnswerBody,TID,Tag
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,python
1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,osx
2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,fonts
3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,photoshop
4,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,469,python
5,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,469,osx
6,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,469,fonts
7,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,469,photoshop
8,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,3040.0,457.0,2008-08-06T03:01:23Z,469.0,12.0,<p>Unfortunately the only API that isn't depre...,469,python
9,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,3040.0,457.0,2008-08-06T03:01:23Z,469.0,12.0,<p>Unfortunately the only API that isn't depre...,469,osx


In [86]:
QAT['QuestionTitleAndBody'] = QAT['QuestionTitle'] + " " + QAT['QuestionBody']

In [87]:
QAT.head()

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback,AID,AnswerUserId,AnswerCreateDate,ParentId,AnswerScore,AnswerBody,TID,Tag,QuestionTitleAndBody
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,python,How can I find the full path to a font from it...
1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,osx,How can I find the full path to a font from it...
2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,fonts,How can I find the full path to a font from it...
3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,photoshop,How can I find the full path to a font from it...
4,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,469,python,How can I find the full path to a font from it...


In [97]:
lem = WordNetLemmatizer()

In [98]:
def lemmatize_text(text):
    return [lem.lemmatize(w) for w in word_tokenize(text)]

In [ ]:
QAT['QuestionTitleAndBodyLemmatized'] = QAT.QuestionTitleAndBody.apply(lemmatize_text)

Look at one for example:

In [103]:
QAT['QuestionTitleAndBodyLemmatized'][2]

['How',
 'can',
 'I',
 'find',
 'the',
 'full',
 'path',
 'to',
 'a',
 'font',
 'from',
 'it',
 'display',
 'name',
 'on',
 'a',
 'Mac',
 '?',
 '<',
 'p',
 '>',
 'I',
 'am',
 'using',
 'the',
 'Photoshop',
 "'s",
 'javascript',
 'API',
 'to',
 'find',
 'the',
 'font',
 'in',
 'a',
 'given',
 'PSD.',
 '<',
 '/p',
 '>',
 '<',
 'p',
 '>',
 'Given',
 'a',
 'font',
 'name',
 'returned',
 'by',
 'the',
 'API',
 ',',
 'I',
 'want',
 'to',
 'find',
 'the',
 'actual',
 'physical',
 'font',
 'file',
 'that',
 'that',
 'font',
 'name',
 'corresponds',
 'to',
 'on',
 'the',
 'disc.',
 '<',
 '/p',
 '>',
 '<',
 'p',
 '>',
 'This',
 'is',
 'all',
 'happening',
 'in',
 'a',
 'python',
 'program',
 'running',
 'on',
 'OSX',
 'so',
 'I',
 'guess',
 'I',
 "'m",
 'looking',
 'for',
 'one',
 'of',
 ':',
 '<',
 '/p',
 '>',
 '<',
 'ul',
 '>',
 '<',
 'li',
 '>',
 'Some',
 'Photoshop',
 'javascript',
 '<',
 '/li',
 '>',
 '<',
 'li',
 '>',
 'A',
 'Python',
 'function',
 '<',
 '/li',
 '>',
 '<',
 'li',
 '>',
 'A

In [104]:
QAT.head()

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback,AID,AnswerUserId,AnswerCreateDate,ParentId,AnswerScore,AnswerBody,TID,Tag,QuestionTitleAndBody,QuestionTitleAndBodyLemmatized
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,python,How can I find the full path to a font from it...,"[How, can, I, find, the, full, path, to, a, fo..."
1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,osx,How can I find the full path to a font from it...,"[How, can, I, find, the, full, path, to, a, fo..."
2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,fonts,How can I find the full path to a font from it...,"[How, can, I, find, the, full, path, to, a, fo..."
3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,469,photoshop,How can I find the full path to a font from it...,"[How, can, I, find, the, full, path, to, a, fo..."
4,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,False,False,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,469,python,How can I find the full path to a font from it...,"[How, can, I, find, the, full, path, to, a, fo..."


In [105]:
QAT.tail()

,QID,QuestionUserId,QuestionCreateDate,QuestionScore,QuestionTitle,QuestionBody,HasError,HasTraceback,AID,AnswerUserId,AnswerCreateDate,ParentId,AnswerScore,AnswerBody,TID,Tag,QuestionTitleAndBody,QuestionTitleAndBodyLemmatized
3182432,40143338,7044980.0,2016-10-19T23:52:27Z,2,Most possible pairs,"<p>Given a list of values, and information on ...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,40143338,python,"Most possible pairs <p>Given a list of values,...","[Most, possible, pair, <, p, >, Given, a, list..."
3182433,40143365,4133131.0,2016-10-19T23:55:49Z,0,Theano operations returning odd results,<p>So I'm trying to learn how to use Theano an...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,40143365,python,Theano operations returning odd results <p>So ...,"[Theano, operation, returning, odd, result, <,..."
3182434,40143365,4133131.0,2016-10-19T23:55:49Z,0,Theano operations returning odd results,<p>So I'm trying to learn how to use Theano an...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,40143365,numpy,Theano operations returning odd results <p>So ...,"[Theano, operation, returning, odd, result, <,..."
3182435,40143365,4133131.0,2016-10-19T23:55:49Z,0,Theano operations returning odd results,<p>So I'm trying to learn how to use Theano an...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,40143365,theano,Theano operations returning odd results <p>So ...,"[Theano, operation, returning, odd, result, <,..."
3182436,40143365,4133131.0,2016-10-19T23:55:49Z,0,Theano operations returning odd results,<p>So I'm trying to learn how to use Theano an...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,40143365,nnet,Theano operations returning odd results <p>So ...,"[Theano, operation, returning, odd, result, <,..."


In [106]:
QAT.to_csv('QAT.csv', index=False)